# Spreadsheet validation with pandas

Once upon a time I was asked to manually, as in just by eyeballing it, validate some bioinformatics data in a spreadsheet to spot rows that did not conform to a given set of rules. Yes, that's a silly thing to ask someone to do. It's a long story, don't ask. 

Anyway, I'd been looking for opportunities to play around with data processing in [pandas](https://pandas.pydata.org/), and decided that taking this request to the next level would be a great way to do that. It's just a first pass, and I'll discuss things I would have done differently at the end, but I'm happy enough with the results to put them up here. 

Enjoy!

## First steps

Let's take care of some imports, load up the spreadsheet, and see what we're working with...

In [12]:
import pandas as pd
import datetime, re
import np
from collections import defaultdict

data = pd.ExcelFile("SGD-Features.xlsx")
sheet = data.parse().fillna('')
sheet

,Row,Primary SGDID,Feature type,Feature name,Gene name,Secondary SGDID,Chromosome,Start coordinate,Stop coordinate,Strand,Sequence version date,Description,Unnamed: 12
0,1,S000001568,ORF,YKL085W,MDH1,L000001045,11,279123,280127,W,1996-07-31,Mitochondrial malate dehydrogenase; catalyzes ...,
1,2,S000001569,ORF,YKL086W,SRX1,,11,278281,278664,W,1996-07-31,Sulfiredoxin; contributes to oxidative stress ...,
2,7,S000001574,ORF,YKL091C,,,11,270650,269718,C,1996-07-31,Putative phosphatidylinositol/phosphatidylchol...,
3,8,S000001575,ORF,YKL092C,BUD2,L000000199,11,269459,266145,C,1996-07-31,GTPase activating factor for Rsr1p/Bud1p; play...,
4,9,S000001576,ORF,YKL093W,MBR1,L000001033,11,264789,265808,W,1996-07-31,protein involved in mitochondrial functions an...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,147,S000001705,ORF,YKL222C,,,11,5620,3503,C,1996-07-31,Protein of unknown function; may interact with...,
80,148,S000001706,ORF,YKL223W,,,11,2389,2721,W,1996-07-31,Dubious open reading frame; unlikely to encode...,
81,149,S000001707,ORF,YKL224C,PAU16,,11,2181,1810,C,1996-07-31,Protein of unknown function; member of the ser...,
82,150,S000001708,ORF,YKL225W,,,11,451,798,W,1996-07-31,Dubious open reading frame; unlikely to encode...,


## We need to apply the following validation rules:
```
1.      Row number: use this when you refer to the rows with errors (mandatory)
2.   	Primary SGDID: a unique identifier for the sequence feature (mandatory)
3.   	Feature type: a phrase that describes the sequence feature   (mandatory
4.   	Feature name: a name that describes the approximate location of the sequence feature (optional)
5.      Gene name: a human-friendly name, following the pattern of 3-4 letters and a number (optional)
6.   	Secondary SGDID: alternate identifiers for the sequence feature  (optional, multiples separated by |)
7.   	Chromosome: the chromosome that the sequence feature is located on (optional)
8.  	Start coordinate: the coordinate number on the chromosome that contains the beginning of the start codon (mandatory)
9.  	Stop coordinate: the coordinate number on the chromosome that  contains the end of the stop codon  (mandatory)
10.  	Strand: whether the ORF is on the Watson or Crick strand (mandatory)
11.  	Sequence version date: date when the genomic sequence was last modified (optional)
12.  	Description: A short summary of the biological role of the ORF (optional)
```

## Or, put another way...

| Field name                 | Is required? | Content must be...
|---                         |---           |---
| Row                        | Y              | a positive integer  
| Primary SGDID              | Y            | unique, an SGID
| Feature type               | Y            |  
| Feature name               | N            | 
| Gene name                  | N            | 3-4 letters followed by a number
| Secondary SGDID            | N            | '\|'-delimited SGIDs
| Chromosome                 | N            | a positive integer
| Start coordinate           | Y            | a positive integer
| Stop coordinate            | Y            | a positive integer
| Strand                     | Y            | "W" or "C"
| Sequence version date      | N            | date
| Description                | N            | 

## Let's write a function for each of those validations

Each one takes a pandas dataframe and a column name, and returns a dataframe with True/False values for each index in the original, corresponding to whether the value for the given column at that index passed validaion. 
This is handy because then multiple validator results can be combined with bitwise boolean operations.

In [13]:
# cell values we will count as "empty"
EMPTY_VALS = ["",np.nan,None]

def exists(df, col, required = True):
    "must not be blank"
    # In this case the `required` option is really just 
    # here for consistency with other validators, 
    # to make testing easier
    if required:
        return ~df[col].isin(EMPTY_VALS)
    else:
        return df[col].apply(lambda x: True)


# helper function controlling whether the calling 
# validator tolerates empty cells
def _with_optionality(df, col, matches, required):
    if required:
        return matches & exists(df, col)
    else:
        return matches | ~exists(df, col)
    
    
def is_positive_int(df, col, required = True):
    "must be a positive integer"
    # corece everything to a number or NaN
    as_nums = pd.to_numeric(df[col], errors='coerce')
    
    # identify positive "numbers" (includes converted strings like "1")
    are_positive = as_nums > 0
    are_ints = as_nums % 1 == 0
    
    # for filtering out non-integers, including strings as described above
    are_actually_numbers = df[col].apply(lambda x: pd.api.types.is_number(x))
    
    matches = are_positive & are_ints & are_actually_numbers
    return _with_optionality(df, col, matches, required)
    
    
def is_unique(df, col, required=True):
    "must be unique"
    # idx,bool: True only for appearance 2+ of a val, not originals
    duplicate_idx = df[col].duplicated()

    # rows with appearance 2+. still doesn't include the original.
    # to get that, we'll need the entire row so we can get...
    duplicate_rows = df[duplicate_idx]

    # SGDID of ^^^
    duplicate_sgdids = duplicate_rows[col]

    # idx, bool: True for all rows containing a duplicated SGDID
    all_duplicates_idx = df[col].isin(duplicate_sgdids)

    matches = ~all_duplicates_idx
    return _with_optionality(df, col, matches, required)


def _matches_regex(df, col, regex):
    return df[col].apply(lambda v: bool(re.match(regex, str(v))))


def is_gene_friendly_name(df, col, required=False):
    "must be 3-4 letters followed by a number"
    matches = _matches_regex(df, col, r'^[A-Z]{3,4}[0-9]+$')
    return _with_optionality(df, col, matches, required)


def is_valid_sgdid(df, col, required=True):
    "must be L or S followed by 9 numbers"
    matches = _matches_regex(df, col, r'^[LS][0-9]{9}$')
    return _with_optionality(df, col, matches, required)


def is_valid_sgdid_list(df, col, required=False):
    "must be a '|'-delimited list of SGDIDs"
    # table of is_valid_sgdid results for each '|'-delimited val, row for row
    matches = df[col].apply(lambda v: is_valid_sgdid(pd.DataFrame({col: str(v).split('|')}), col))
    
    # rows with no False validity booleans
    matches = matches.fillna(True).all(axis=1)
    return _with_optionality(df, col, matches, required)


def is_valid_strand(df, col, required=True):
    "must be 'W' or 'C'"
    matches = df[col].isin(['C','W'])
    return _with_optionality(df, col, matches, required)


def is_valid_date(df, col, required=True):
    "must be a valid date in YYYY-MM-DD format"
    def check_date(date_str):
        try:
            datetime.datetime.strptime(str(date_str), '%Y-%m-%d')
        except ValueError:
            return False
        else:
            return True
        
    matches = df[col].apply(lambda v: check_date(v))
    return _with_optionality(df, col, matches, required)
    

## Tests for the validators

Before applying validators to the spreadsheet, let's include some tests to make sure they work as expected (plus the tests themselves can provide readers with more insight into each validator's intended function).

In [14]:
validator_tests = {
    exists: (
        [1,    "a",  " " ],  # input
        [True, True, True]   # expected output
    ),
    is_positive_int: (
        [1,    "1",   1.5,   -1,  ],
        [True, False, False, False]
    ),
    is_unique: (
        [1,     2,    'a',   'b',  1,     'a'   ],
        [False, True, False, True, False, False ]
    ),
    is_gene_friendly_name: (
        ["ABC1", "ABCD2", "ABCD22", "ABC2a", "ABC",  'AB1',  '1',   'abc1'],
        [True,   True,    True,     False,   False,  False,  False, False ]
    ),
    is_valid_sgdid: (
        ["L123456789", "S123456789", "A123456789", "L12345678", "L1234567890"],
        [True,         True,         False,        False,       False         ]
    ),
    is_valid_sgdid_list: (
        ["L123456789", "S123456789|L123456789", "S123456789|A123456789", "S123456789,L123456789", ],
        [True,         True,                    False,                   False,                   ]
    ),
    is_valid_strand: (
        ['C',   'W',  'c',   'X',  ],
        [True, True, False, False ]
    ),
    is_valid_date: (
        ['2020-04-01', '2020-90-01',  '2020-04-90',  '04-09-2020',  ],
        [True,         False,         False,         False,         ]
    ),
}
for f, (given, expected) in validator_tests.items():
    def run_test(f, given, expected, required):
        df = pd.DataFrame({'vals': given})
        res = list(f(df, 'vals', required = required))
        label = "{} ({})".format(f.__name__, "required" if required else "optional")
        if res == expected:
            print("{}... PASS".format(label))
        else:
            report = pd.DataFrame({'Given': given, 'Expected': expected, 'Got': res})
            print("{}... FAIL\n    {}".format(
                label,
                '\n    '.join(report.to_string(index=False).split('\n'))
            ))
        
    given += EMPTY_VALS
    expected_when_required = expected + [False] * len(EMPTY_VALS)
    expected_when_optional = expected + [True]  * len(EMPTY_VALS)
    
    run_test(f, given, expected_when_required, True)
    run_test(f, given, expected_when_optional, False)


exists (required)... PASS
exists (optional)... PASS
is_positive_int (required)... PASS
is_positive_int (optional)... PASS
is_unique (required)... PASS
is_unique (optional)... PASS
is_gene_friendly_name (required)... PASS
is_gene_friendly_name (optional)... PASS
is_valid_sgdid (required)... PASS
is_valid_sgdid (optional)... PASS
is_valid_sgdid_list (required)... PASS
is_valid_sgdid_list (optional)... PASS
is_valid_strand (required)... PASS
is_valid_strand (optional)... PASS
is_valid_date (required)... PASS
is_valid_date (optional)... PASS


## Now we're ready to associate tests with columms
...plus a function to run them. The magic happens in that `for test in tests` loop where, because pandas lets us run the test on an entire column at once, a simple bitwise NOT (`~`) on the results lets us easily separate out the failures.

In [15]:
def run_tests(tests_by_col):
    # See below for example tests_by_col data
    # There's probably a way to do this in pandas with MultiIndex?
    fails_by_row_by_col = defaultdict(lambda: defaultdict(list))

    for col, (required, tests) in tests_by_col.items():
        col_key = "{} field {}".format("required" if required else "optional", col)
        for test in tests:
            valid_idxs = test(sheet, col, required = required)
            fail_idxs  = ~valid_idxs
            fail_items = sheet[fail_idxs]
            if not fail_items.empty:
                for row_key in list(fail_items[['Row',col]].apply(lambda df: "Row {} with value '{}'".format(df['Row'], df[col]), axis=1)):
                    fails_by_row_by_col[col_key][row_key].append(test.__doc__)
            
    for col_key, fails_by_row in fails_by_row_by_col.items():
        print("In {}...".format(col_key))
        for row_key, failed_tests in fails_by_row.items():
            print ("  {} failed validation:\n    - {}".format(row_key, '\n    - '.join(failed_tests)))
        print('')

# Now we associate each column in the spreadsheet with whether it's required, and the relevant validations
tests_by_col = {
    'Row': (
        True,                 # required
        [is_positive_int,],   # tests
    ),
    'Primary SGDID': (
        True,
        [is_unique, is_valid_sgdid],
    ),
    
    'Feature type': (
        True,
        [exists],
    ),
    
    'Gene name': (
        False,
        [is_gene_friendly_name],
    ),
    
    'Secondary SGDID': (
        False,
        [is_valid_sgdid_list],
    ),
    'Chromosome': (
        False,
        [is_positive_int],
    ),
    'Start coordinate': (
        True,
        [is_positive_int],
    ),
    'Stop coordinate': (
        True,
        [is_positive_int],
    ),
    'Strand': (
        True,
        [is_valid_strand],
    ),
    'Sequence version date': (
        False,
        [is_valid_date],
    ),
}

## And the results!
_See "Final thoughts" below for why the lone blank SGDID on row 114 fails the uniqueness test._

In [16]:
run_tests(tests_by_col)

In required field Primary SGDID...
  Row 96 with value 'S000001654' failed validation:
    - must be unique
  Row 97 with value 'S000001654' failed validation:
    - must be unique
  Row 114 with value '' failed validation:
    - must be unique
    - must be L or S followed by 9 numbers

In optional field Gene name...
  Row 17 with value 'YKL099C' failed validation:
    - must be 3-4 letters followed by a number
  Row 55 with value '2013-10-01 00:00:00' failed validation:
    - must be 3-4 letters followed by a number
  Row 80 with value 'RPS27A' failed validation:
    - must be 3-4 letters followed by a number
  Row 81 with value 'YKL157W' failed validation:
    - must be 3-4 letters followed by a number

In optional field Secondary SGDID...
  Row 17 with value 'chromosome 11' failed validation:
    - must be a '|'-delimited list of SGDIDs

In required field Strand...
  Row 17 with value '255720' failed validation:
    - must be 'W' or 'C'

In optional field Sequence version date...
 

## Final thoughts

In retrospect I think adding a `required` option to each validator was an un-necessary complication. If I were to do this again, I would make it so if a column is marked as required then the `exists` validator is run on it, and if that fails then it's reported and we don't bother with other validators. Using required-ness to control each validator's tolerance of empty cells is what caused the one blank SGDID on line 114 to be reported as not unique above. Not strictly accurate, but it highlights the error well enough for a first attempt.